# Desarrollo modelo epidemiológico -  A. Bassi


In [1]:
import numpy as np
import pandas as pd
import toml
import matplotlib.pyplot as plt
import time
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator

import epyestim
import epyestim.covid19 as covid19
from epyestim.distributions import discretise_gamma
from datetime import timedelta

# rpy2 imports
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()
# R library
global eps
eps = importr("EpiEstim")
import sys
from pathlib import Path
sys.path.insert(1, '../')
from src2.models.cv19sim import cv19sim

/home/samuel/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:
#Pop-up plots
import platform
OS = platform.system()
if OS == 'Linux' or OS == 'Darwin':
    %matplotlib tk    
elif OS == 'Windows':
    %matplotlib qt    

## Valores iniciales
$ \beta_e = \beta\frac{S}{N}$

$ I_0 = \frac{C_0}{\beta_e}$

$ E_0 = \frac{C_0}{\delta +\sigma}$

$ E_0 = \frac{C_0}{\delta +\sigma}$

$\delta = \frac{1}{2}\sqrt{4\sigma\beta + (\sigma-\gamma)^2} - \sigma - \gamma$

Entonces:
$ E_0 = \frac{C_0}{\frac{1}{2}\sqrt{4\sigma\beta + (\sigma-\gamma)^2}- \gamma}$

Donde $C_0$ son los infectados nuevos diarios $(I_d)$ el día 0
### Data Real
Cuando tenemos data real tenemos valores tanto de $I_0$ como de $C_0$, por lo que para lograr una partida suave es necesario usar uno para calcular el otro, o buscar un valor promedio que minimice el error entre ambos. Ese trabajo quedará para un poco más adelante

## Calculo de pendientes en escala logarítmica

In [3]:
def getdelta(y,x=None,scale='linear'):
    if scale == 'linear':
        y = np.log(y)
    delta = np.diff(y)
    return delta

# Generación de datos sintéticos
Trabajaremos con los infectados nuevos diarios, dado que esos son los datos que se miden en la realidad. El objetivo será tratar de reconstruir los parámetros originales. Truncaremos los datos para reducir el efecto de estabilización inicial. Partiremos con datos desde que los infectados activos superen los 100 casos. 
Por ahora despreciaremos los efectos del subreporte y del ruido de medición.

In [4]:
cfgfile = '../config_files/SEIR2.toml'
cfg = toml.load(cfgfile) # no es necesario, pero es util para ver el archivo de configuracion

In [5]:
beta = 0.4
I0 = 10
mu = 1
tE_I = 3
tI_R = 5
sigma = 1/tE_I
gamma = 1/tI_R

In [6]:
sims = cv19sim(cfg,beta = beta,mu=mu,I=I0,I_d=I0*beta,tE_I = tE_I,tI_R=tI_R,t_end=250)

In [7]:
sims.integrate()

In [8]:
plt.plot(sims.sims[0].I_d,label='Nuevos Infectados Diarios')
plt.legend(loc=0)

In [9]:
# Tiempo de los datos inicial
t0 = np.where(sims.sims[0].I >= 100)[0][0]

In [10]:
I_d = sims.sims[0].I_d[t0:]

### Cálculo R analítico
$R_e = \beta \frac{S}{N}T_{IR}$

In [11]:
Re_analitico = beta*sims.sims[0].S/sims.sims[0].N*tI_R
Re_analitico = Re_analitico[t0:] # Correccion temporal

# Trabajo con la data sintética

In [12]:
plt.scatter(range(len(I_d)),I_d,label='data')
#plt.plot(t[1:],dI_d,label='dI_d')

### Calculo de $\delta$

In [13]:
delta = getdelta(I_d)

In [14]:
plt.plot(range(len(delta)),delta,label='delta')
plt.title('Pendiente Infectados diarios escala log')
plt.legend(loc=0)

In [15]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")

axs2.set_ylabel('Delta',color='tab:red')
axs2.plot(range(len(delta)),delta,label='delta',color='tab:red')

axs2.tick_params(axis='y', labelcolor='tab:red')

axs = axs2.twinx()
axs.plot(range(len(I_d)),np.log(I_d),color='tab:blue',label='Diarios',linestyle='solid')
#axs.plot(sim.t,sim.I,color='tab:blue',label='Activos')
axs.set_ylabel('Infectados',color='tab:blue')
axs.tick_params(axis='y', labelcolor='tab:blue')

#axs2.set_xlim(0,200)
fig.suptitle('Delta e Id')
axs.legend(loc=1)
axs2.legend(loc=3)
fig.show()

## Tiempo Serial
Uno de los mayores desafíos es tener un tiempo serial bien calculado.
Analíticamente la definición del tiempo serial es:  
$ T_s = T_{EI} + T_{IR} $  
Donde $T_{EI}$ y $T_{IR}$ son las medias de las distribuciones de tiempos de incubación e infecciosidad respectivamente. 
Para este análisis trabajaremos primero con los tiempos seriales de la misma simulación.

In [16]:
Ts = tE_I + tI_R

## Cálculo del R efectivo a partir de $\delta$

In [17]:
# R delta
Re_delta = []
for i in range(len(delta)):
    Re_delta.append(1 + delta[i]*Ts + sigma*gamma/(sigma + gamma)*(delta[i]*Ts)**2)
Re_delta=np.array(Re_delta)

# R delta aprox
Re_delta_a = 1 + delta*Ts

## Estudios de Re: Comparación R analítico, R delta y R delta aprox

In [18]:
plt.plot(range(len(Re_analitico[1:])),Re_analitico[1:],label='Analitico')
plt.plot(range(len(Re_delta)),Re_delta,label='delta')
plt.plot(range(len(Re_delta_a)),Re_delta_a,label='delta aprox')
plt.axhline(1,color='grey',linestyle='dashed')
plt.title('Estudios Re')
plt.legend(loc=0)
plt.show()

In [19]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")


axs2.set_ylabel('Delta',color='tab:red')
axs2.plot(range(len(Re_analitico[1:])),Re_analitico[1:],label='Analitico')
axs2.plot(range(len(Re_delta)),Re_delta,label='delta')
axs2.plot(range(len(Re_delta_a)),Re_delta_a,label='delta aprox')
axs2.axhline(1,color='grey',linestyle='dashed')
axs2.tick_params(axis='y', labelcolor='tab:red')

axs = axs2.twinx()
axs.plot(range(len(I_d)),np.log(I_d),color='tab:blue',label='Diarios',linestyle='solid')
axs2.axvline(np.where(I_d==max(I_d))[0][0],color='grey',linestyle='dashed')
#axs.plot(sim.t,sim.I,color='tab:blue',label='Activos')
axs.set_ylabel('Infectados',color='tab:blue')
axs.tick_params(axis='y', labelcolor='tab:blue')

#axs2.set_xlim(0,200)
fig.suptitle('Re e Id')
axs.legend(loc=1)
axs2.legend(loc=3)
fig.show()

plt.show()

### Error R

In [20]:
P = 1
err_delta =  ((P*Re_delta - P*Re_analitico[1:])**2).mean() 
err_delta_a =  ((P*Re_delta_a - P*Re_analitico[1:])**2).mean() 
print('Error calculo Delta: '+str(err_delta))
print('Error calculo Delta aprox: '+str(err_delta_a))

Error calculo Delta: 0.0038381003424130386
Error calculo Delta aprox: 0.014974080572622957


## Calculo de beta
$ \beta(t) = R_e \frac{N}{S(t)} \gamma $

In [21]:
beta_Re_delta = gamma*Re_delta*sims.sims[0].N/sims.sims[0].S[t0+1:]
beta_Re_delta_a = gamma*Re_delta_a*sims.sims[0].N/sims.sims[0].S[t0+1:]
beta_Re_analitico = gamma*Re_analitico*sims.sims[0].N/sims.sims[0].S[t0:]

In [22]:
plt.plot(range(len(beta_Re_delta)),beta_Re_delta,label='Delta')
plt.plot(range(len(beta_Re_delta_a)),beta_Re_delta_a,label='Delta aprox')
plt.plot(range(len(beta_Re_analitico)),beta_Re_analitico,label='beta analitico')
plt.axhline(beta,label='Real',linestyle='dashed')
plt.legend(loc=0)
plt.show()

# Análisis de sensibilidad de tiempo serial
Para hacer este estudio utilizaremos los mismos datos sintéticos y calcularemos el Re y beta utilizando distintos gammas y sigmas para ver como crece el error a medida que nos alejamos de los valores correctos

In [23]:
tE_I_s = np.arange(2,6,0.1)
tI_R_s = np.arange(2,6,0.1)
Ts_s = np.array([[i+j for i in tE_I_s] for j in tI_R_s])

In [24]:
sigma_s = 1/tE_I_s
gamma_s = 1/tI_R_s

In [25]:
P = 1
# R delta
Re_delta_s_a = np.zeros(np.shape(Ts_s), dtype=object)  
Re_delta_s = np.zeros(np.shape(Ts_s), dtype=object)
error_s = np.zeros(np.shape(Ts_s))
error_s_a = np.zeros(np.shape(Ts_s))
for i in range(len(sigma_s)):
    for j in range(len(gamma_s)):
        aux = []
        aux_a = []
        for k in range(len(delta)):
            aux.append(1 + delta[k]*(1/sigma_s[i]+1/gamma_s[j]) + sigma_s[i]*gamma_s[j]/(sigma_s[i] + gamma_s[j])*(delta[k]*(1/sigma_s[i]+1/gamma_s[j]))**2)
            aux_a.append(1 + delta[k]*(1/sigma_s[i]+1/gamma_s[j]))
        Re_delta_s[i,j] = np.array(aux)
        Re_delta_s_a[i,j]= np.array(aux_a)
        error_s[i,j] = ((P*Re_delta_s[i,j] - P*Re_analitico[1:])**2).mean()
        error_s_a[i,j] = ((P*Re_delta_s_a[i,j] - P*Re_analitico[1:])**2).mean()
            

In [26]:
P = 1
# R delta
Re_delta_s_a = np.zeros(np.shape(Ts_s), dtype=object)  
Re_delta_s = np.zeros(np.shape(Ts_s), dtype=object)
error_s = np.zeros(np.shape(Ts_s))
error_s_a = np.zeros(np.shape(Ts_s))
for i in range(len(sigma_s)):
    for j in range(len(gamma_s)):
        aux = []
        aux_a = []
        for k in range(len(delta)):
            aux.append(1 + delta[k]*(1/sigma_s[i]+1/gamma_s[j]) + sigma_s[i]*gamma_s[j]/(sigma_s[i] + gamma_s[j])*(delta[k]*(1/sigma_s[i]+1/gamma_s[j]))**2)
            aux_a.append(1 + delta[k]*(1/sigma_s[i]+1/gamma_s[j]))
        Re_delta_s[i,j] = np.array(aux)
        Re_delta_s_a[i,j]= np.array(aux_a)
        error_s[i,j] = ((P*Re_delta_s[i,j] - P*Re_delta[:])**2).mean()
        error_s_a[i,j] = ((P*Re_delta_s_a[i,j] - P*Re_delta[:])**2).mean()
            

### Analysis

In [27]:
# Contour Plot
levels = np.arange(0,0.121,0.001)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I_s,tI_R_s,error_s,levels)

cp2 = ax.contour(tE_I_s,tI_R_s,error_s,[0,np.min(error_s)+0.001,0.12],colors='white',linestyles='dashed')
cp2 = ax.contour(tE_I_s,tI_R_s,error_s,[0,np.min(error_s)+0.0000001,0.12],colors='white',linestyles='dashed')
#
#ax.clabel(cp2, inline=1, fontsize=20,fmt=str(l))

fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Error calculo completo')
ax.set_xlabel('Incubation period')
ax.set_ylabel('Infectious period')
plt.show()

In [29]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I_s,tI_R_s,error_s_a) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Error calculo aprox')
ax.set_xlabel('Incubation period')
ax.set_ylabel('Infectious period')
plt.show()

In [31]:
l = 3
j = 30 # gamma
for i in range(int(len(sigma_s)/l)):
    plt.plot(range(len(Re_delta_s[i*l,j])),Re_delta_s[i*l,j],label='tE_I='+str(tE_I_s[i*l]))
plt.plot(range(len(Re_analitico)),Re_analitico,label='analitico',linestyle='dashed',color='grey')
plt.legend(loc=0)

In [33]:
Re_analitico[0]

1.9892594347214363

# Error vs R0

In [146]:
beta2 = list(np.arange(0.2,0.6,0.01))
R0 = np.array(beta2)*tI_R

In [151]:
sims2 = cv19sim(cfg,beta = beta2,mu=mu,I=I0,I_d=I0*beta,tE_I = tE_I,tI_R=tI_R,t_end=250)
sims2.integrate()

### Calculo de R por delta

In [152]:
t0=15

In [153]:
Re_analitico2 = [beta2[i]*sims2.sims[i].S[t0:-1]/sims2.sims[i].N*tI_R for i in range(len(beta2))]


In [154]:
delta2 = [getdelta(sims2.sims[i].I_d[t0:]) for i in range(len(beta2))]

In [155]:
len(delta2[0])

234

In [156]:
len(Re_analitico2[0])

234

In [157]:
# R delta
Re_delta2 = []
for j in delta2:
    aux = []
    for i in range(len(j)):
        aux.append(1 + j[i]*Ts + sigma*gamma/(sigma + gamma)*(j[i]*Ts)**2)
    Re_delta2.append(aux)
Re_delta2=np.array(Re_delta2)

# R delta aprox
#Re_delta_a = 1 + delta*Ts

In [158]:
plt.plot(Re_delta2[-1])

In [159]:
# Error
err2 = []
for i in range(len(beta2)):
    err2.append((Re_analitico2[i][0:100]-Re_delta2[i][0:100]).mean())

In [162]:
plt.plot(R0,err2)
plt.title('Error calculo R vs R0')
plt.xlabel('R0')
plt.ylabel('Error')
plt.plot()

[]

# To Do:
[ ] Comprobar el orden de las variables en los arreglos sigma-gamma  
[ ] Verificar el calculo de R porque no me están calzando bien los R con las magnitudes de los otros.
[ ] Grid-plot  


[ ] Estudiar cuanto aumenta el error a medida que cambian el tiempo serial, gamma y sigma 

[x] Calcular Beta desde el R  
[ ] Reconstruir las curvas con los parámetros obtenidos  
[ ] Tratar de compatibilizar los cálculos de R de Felipe y de la nueva librería, y compararlos con el cálculo analítico  

[ ] Revisar calculo del error entre los cálculos de R para el contourplot.  
[ ] Revisar calculo del error entre los cálculos de R. 